# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

In [ ]:
!sed -E 's/(=[^=]+)=.*$/\1/' multiff_analysis/environment.yml > multiff_analysis/environment_nobuild.yml

In [ ]:
!sed -E 's/(=[^=]+)=.*$/\1/' multiff_analysis/pgam_environment.yml > multiff_analysis/pgam_environment_nobuild.yml

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from neural_data_analysis.neural_analysis_tools.glm_tools import glm_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases, plot_glm_fit, glm_on_multiff
import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
reload(pn_helper_class)

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = False
y_data_exists_ok = True

pn = pn_aligned_by_seg.PlanningAndNeuralSegmentAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
# pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)
pn.rebin_data_in_new_segments(segment_duration=2, rebinned_max_x_lag_number=2)

for col in ['cur_vis', 'nxt_vis', 'cur_in_memory', 'nxt_in_memory']:
    pn.rebinned_y_var[col] = (pn.rebinned_y_var[col] > 0).astype(int)

# exp

In [ ]:
pn.monkey_information['dt'].min()

## speed

In [ ]:
reload(process_monkey_information)

In [ ]:
m_df = pn.monkey_information.copy()
out = process_monkey_information.compute_kinematics_loclin(m_df, time_col='time', x_col='monkey_x', y_col='monkey_y', theta_col='monkey_angle',
                              window_s=0.040, use_theta='auto')

In [ ]:
pn.monkey_information[['monkey']]

In [ ]:
out[out['dt'] < 0.01]

In [ ]:
i = 1945
out.loc[i:i+30,['dt', 'monkey_speed', 'speed', 'monkey_ddv', 'accel', 'monkey_angle', 'monkey_dw', 'ang_speed', 'monkey_ddw', 'ang_accel']]

In [ ]:
# import numpy as np

# def _tricube(u):
#     u = np.clip(np.abs(u), 0, 1)
#     return (1 - u**3)**3

# def _local_linear_slope(time, z, window_s=0.040, min_pts=5, ridge=1e-8):
#     """
#     Locally weighted linear regression of z(t) per sample.
#     Returns dz/dt at each t_k. Handles irregular sampling.
#     """
#     t = np.asarray(time, float)
#     z = np.asarray(z, float)
#     n = t.size
#     half = 0.5 * window_s
#     dzdt = np.zeros(n, float)

#     for k in range(n):
#         t0, t1 = t[k] - half, t[k] + half
#         idx = np.flatnonzero((t >= t0) & (t <= t1))
#         widen = 1.0
#         while idx.size < min_pts and widen <= 10:
#             widen *= 1.5
#             idx = np.flatnonzero((t >= t[k] - half*widen) & (t <= t[k] + half*widen))
#         tk = t[idx]
#         tc = tk - t[k]
#         w = _tricube(np.abs(tc) / max(half, 1e-9))
#         X = np.column_stack([np.ones_like(tc), tc])
#         XtW = X.T * w
#         G = XtW @ X + ridge * np.eye(2)
#         b  = XtW @ z[idx]
#         coeff = np.linalg.solve(G, b)  # [intercept, slope]
#         dzdt[k] = coeff[1]
#     return dzdt

# def local_linear_velocity(time, x, y, window_s=0.040, min_pts=5, ridge=1e-8):
#     """
#     Locally weighted linear regression of x(t), y(t) per sample.
#     Returns vx, vy, speed.
#     """
#     vx = _local_linear_slope(time, x, window_s=window_s, min_pts=min_pts, ridge=ridge)
#     vy = _local_linear_slope(time, y, window_s=window_s, min_pts=min_pts, ridge=ridge)
#     speed = np.hypot(vx, vy)
#     return vx, vy, speed

# def compute_kinematics_loclin(df, time_col='time', x_col='x', y_col='y', theta_col='theta',
#                               window_s=0.040, use_theta='auto'):
#     """
#     Adds: vx, vy, speed, accel (tangential), ang_speed, ang_accel.
#     - use_theta: 'auto' | 'measured' | 'velocity'
#         'auto'      -> use measured theta if available & finite; else velocity-derived
#         'measured'  -> force use theta_col (will fallback if missing)
#         'velocity'  -> ignore theta_col, derive heading from vx,vy
#     """
#     out = df.copy()
#     t = out[time_col].to_numpy()
#     x = out[x_col].to_numpy()
#     y = out[y_col].to_numpy()

#     # linear velocity via local-linear slopes
#     vx, vy, speed = local_linear_velocity(t, x, y, window_s=window_s)
#     out['vx'] = vx
#     out['vy'] = vy
#     out['speed'] = speed

#     # tangential acceleration (project a onto v)
#     ax = _local_linear_slope(t, vx, window_s=window_s)
#     ay = _local_linear_slope(t, vy, window_s=window_s)
#     a_t = (vx*ax + vy*ay) / np.maximum(speed, 1e-9)
#     out['accel'] = a_t

#     # choose heading source
#     heading_from_v = np.arctan2(vy, vx)
#     heading_src = 'velocity'
#     if use_theta in ('auto', 'measured') and theta_col in out.columns:
#         theta = out[theta_col].to_numpy().astype(float)
#         if np.isfinite(theta).any():
#             heading = np.unwrap(theta)
#             heading_src = 'measured'
#         else:
#             heading = np.unwrap(heading_from_v)
#     elif use_theta == 'velocity':
#         heading = np.unwrap(heading_from_v)
#     else:
#         heading = np.unwrap(heading_from_v)

#     # angular kinematics via local-linear slopes for robustness
#     ang_speed = _local_linear_slope(t, heading, window_s=window_s)
#     ang_accel = _local_linear_slope(t, ang_speed, window_s=window_s)

#     out['ang_speed'] = ang_speed
#     out['ang_accel'] = ang_accel
#     out['heading_source'] = heading_src  # handy for QA
#     return out


## binning

In [ ]:
import numpy as np
import pandas as pd

def _interpolate_to_times(time, value, target_times):
    t = np.asarray(time, float)
    v = np.asarray(value, float)
    tau = np.asarray(target_times, float)
    return np.interp(tau, t, v)

def _bin_timeseries_weighted(time, values, bin_edges, how='mean'):
    t = np.asarray(time, float)
    V = np.asarray(values, float)
    if V.ndim == 1:
        V = V[:, None]

    n = t.size
    assert n == V.shape[0] and n >= 2, 'need ≥2 samples with matching values'

    dt = np.diff(t, prepend=t[0])
    if n > 1:
        dt[0] = dt[1]

    bin_idx = np.searchsorted(bin_edges, t, side='right') - 1
    n_bins = len(bin_edges) - 1

    out_sum = np.zeros((n_bins, V.shape[1]), float)  # ∑ v·Δt
    out_w   = np.zeros(n_bins, float)                # ∑ Δt

    valid = (bin_idx >= 0) & (bin_idx < n_bins) & np.isfinite(dt)
    np.add.at(out_sum, bin_idx[valid], (V[valid] * dt[valid, None]))
    np.add.at(out_w,   bin_idx[valid], dt[valid])

    if how == 'mean':
        with np.errstate(invalid='ignore', divide='ignore'):
            out = out_sum / out_w[:, None]
        out[~np.isfinite(out)] = np.nan
        return out.squeeze()
    elif how == 'sum':
        return out_sum.squeeze()
    else:
        raise ValueError('how must be "mean" or "sum"')

def _bin_positions(df, bin_edges, time_col='time', x_col='x', y_col='y', method='center'):
    """
    method:
      'center' -> interpolate x,y to bin centers (good for GLM covariates)
      'start'  -> interpolate to left edges
      'end'    -> interpolate to right edges
      'mean'   -> time-weighted mean position within bin (occupancy centroid)
    """
    t = df[time_col].to_numpy()
    x = df[x_col].to_numpy()
    y = df[y_col].to_numpy()

    n_bins = len(bin_edges) - 1
    centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    if method in ('center', 'start', 'end'):
        if method == 'center':
            query_t = centers
        elif method == 'start':
            query_t = bin_edges[:-1]
        else:
            query_t = bin_edges[1:]
        xb = _interpolate_to_times(t, x, query_t)
        yb = _interpolate_to_times(t, y, query_t)
        return pd.DataFrame({'t_bin': query_t, 'x_bin': xb, 'y_bin': yb})

    if method == 'mean':
        dt = np.diff(t, prepend=t[0])
        if t.size > 1:
            dt[0] = dt[1]
        bin_idx = np.searchsorted(bin_edges, t, side='right') - 1
        valid = (bin_idx >= 0) & (bin_idx < n_bins) & np.isfinite(dt)

        x_sum = np.zeros(n_bins, float)
        y_sum = np.zeros(n_bins, float)
        w_sum = np.zeros(n_bins, float)

        np.add.at(x_sum, bin_idx[valid], x[valid] * dt[valid])
        np.add.at(y_sum, bin_idx[valid], y[valid] * dt[valid])
        np.add.at(w_sum, bin_idx[valid], dt[valid])

        with np.errstate(invalid='ignore', divide='ignore'):
            x_mean = x_sum / w_sum
            y_mean = y_sum / w_sum
        x_mean[~np.isfinite(x_mean)] = np.nan
        y_mean[~np.isfinite(y_mean)] = np.nan

        return pd.DataFrame({'t_bin': centers, 'x_bin': x_mean, 'y_bin': y_mean})

    raise ValueError("method must be 'center', 'start', 'end', or 'mean'")

def make_binned_features(df,
                         bin_width=0.2,
                         start=None,
                         stop=None,
                         # required columns
                         time_col='time',
                         x_col='x',
                         y_col='y',
                         vx_col='vx',
                         vy_col='vy',
                         speed_col='speed',
                         ang_speed_col='ang_speed',
                         spike_col='spike_count',
                         # position binning choice
                         pos_method='center'):
    """
    Build fixed-width bin edges and return one row per bin with:
      - position per bin (interpolated or time-weighted mean)
      - time-weighted mean kinematics (vx, vy, speed, ang_speed)
      - summed spikes
      - offset_log_bin for Poisson GLM

    Parameters
    ----------
    df : DataFrame
        Contains continuous-time samples with columns listed below.
        Kinematics should already be computed at native resolution
        (e.g., via local-linear velocity).
    bin_width : float
        Bin size in seconds (e.g., 0.2 for 200 ms).
    start, stop : float, optional
        Start/stop times for binning. Defaults to data min/max.
    time_col, x_col, y_col, vx_col, vy_col, speed_col, ang_speed_col, spike_col : str
        Column names in `df`.
    pos_method : {'center','start','end','mean'}
        How to define per-bin position:
          - 'center'/'start'/'end': interpolate x,y to those times
          - 'mean': time-weighted mean position within the bin

    Returns
    -------
    binned : DataFrame with columns
        t_bin, x_bin, y_bin,
        vx_bin, vy_bin, speed_bin, ang_speed_bin,
        spike_count_bin, offset_log_bin
    """
    t = df[time_col].to_numpy()
    if start is None:
        start = float(t[0])
    if stop is None:
        stop = float(t[-1])

    # Build edges (include small epsilon to catch the last bin)
    edges = np.arange(start, stop + bin_width*0.5, bin_width)
    centers = 0.5 * (edges[:-1] + edges[1:])

    # Positions per bin (interpolated or time-weighted mean)
    pos_binned = _bin_positions(
        df, edges,
        time_col=time_col, x_col=x_col, y_col=y_col,
        method=pos_method
    )

    # Kinematics: time-weighted mean within each bin
    vx_bin = _bin_timeseries_weighted(t, df[vx_col].to_numpy(), edges, how='mean')
    vy_bin = _bin_timeseries_weighted(t, df[vy_col].to_numpy(), edges, how='mean')
    speed_bin = _bin_timeseries_weighted(t, df[speed_col].to_numpy(), edges, how='mean')
    ang_speed_bin = _bin_timeseries_weighted(t, df[ang_speed_col].to_numpy(), edges, how='mean')

    # Spikes: sum counts in bin
    spike_count_bin = _bin_timeseries_weighted(t, df[spike_col].to_numpy(), edges, how='sum')

    # Assemble one row per bin; ensure t_bin matches positions' time axis
    # (pos_binned['t_bin'] is centers for 'center'/'mean', left/right edges for others)
    binned = pd.DataFrame({
        't_bin': pos_binned['t_bin'].to_numpy(),
        'x_bin': pos_binned['x_bin'].to_numpy(),
        'y_bin': pos_binned['y_bin'].to_numpy(),
        'vx_bin': vx_bin,
        'vy_bin': vy_bin,
        'speed_bin': speed_bin,
        'ang_speed_bin': ang_speed_bin,
        'spike_count_bin': spike_count_bin,
        'offset_log_bin': np.log(bin_width),
    })

    # If you want all features aligned strictly at bin centers, you can
    # enforce t_bin = centers here (uncomment next line):
    # binned['t_bin'] = centers

    return binned


In [ ]:
# df_native has: time,x,y,vx,vy,speed,ang_speed,spike_count at native resolution
b200 = make_binned_features(
    df_native,
    bin_width=0.2,
    time_col='time',
    x_col='x', y_col='y',
    vx_col='vx', vy_col='vy',
    speed_col='speed',
    ang_speed_col='ang_speed',
    spike_col='spike_count',
    pos_method='center'   # or 'mean' for occupancy centroid
)

# then build your GLM design by z-scoring b200[['speed_bin', ...]]
# and using b200['offset_log_bin'] for the Poisson offset.


## more binning

In [ ]:
import numpy as np
import pandas as pd

def bin_timeseries_weighted(time, values, bin_edges, how='mean'):
    """
    Bin irregularly-sampled time series into fixed time bins,
    using **time-weighted integration**.

    Parameters
    ----------
    time : array-like, shape (n_samples,)
        Sample timestamps (must be strictly increasing).
    values : array-like, shape (n_samples,) or (n_samples, n_features)
        Values sampled at each timestamp. Can be scalar or vector.
    bin_edges : array-like, shape (n_bins+1,)
        Monotonic array of bin edges in seconds. 
        Bins are [edge[i], edge[i+1]).
    how : {'mean', 'sum'}
        'mean' → returns the **time-weighted average** of the values
                 in each bin (∫v dt / Δt).
        'sum'  → returns the **time integral** (∫v dt) in each bin.

    Returns
    -------
    binned_values : ndarray, shape (n_bins,) or (n_bins, n_features)
        The aggregated values per bin.

    Notes
    -----
    - Weights each sample by how long it "lasted" (Δt since previous sample).
    - This prevents bias from irregular sampling (e.g., 3 ms bursts vs 16 ms gaps).
    - Example: with 'mean', speed_bin = distance traveled / bin_duration.
    """
    t = np.asarray(time, float)
    V = np.asarray(values, float)
    if V.ndim == 1:
        V = V[:, None]

    n = t.size
    assert n == V.shape[0] and n >= 2, 'need ≥2 samples with matching values'

    # Duration that each sample value was held (since last sample).
    # This is the "weight" for time averaging.
    dt = np.diff(t, prepend=t[0])
    if n > 1:
        dt[0] = dt[1]   # make the first weight not zero

    # Assign each sample to a bin by its timestamp
    bin_idx = np.searchsorted(bin_edges, t, side='right') - 1
    n_bins = len(bin_edges) - 1

    out_sum = np.zeros((n_bins, V.shape[1]), float)  # ∑(v * Δt) per bin
    out_w   = np.zeros(n_bins, float)                # ∑Δt per bin (denominator for mean)

    # Only keep valid indices (within bin range)
    valid = (bin_idx >= 0) & (bin_idx < n_bins) & np.isfinite(dt)

    # Weighted accumulation: add v*dt to sum, and dt to weight
    np.add.at(out_sum, bin_idx[valid], (V[valid] * dt[valid, None]))
    np.add.at(out_w,   bin_idx[valid], dt[valid])

    if how == 'mean':
        # Divide weighted sum by total time in bin
        with np.errstate(invalid='ignore', divide='ignore'):
            out = out_sum / out_w[:, None]
        out[~np.isfinite(out)] = np.nan
        return out.squeeze()
    elif how == 'sum':
        # Return ∑(v * Δt) (time integral) — useful for spike counts
        return out_sum.squeeze()
    else:
        raise ValueError('how must be "mean" or "sum"')


def bin_kinematics_for_glm(df,
                           bin_width=0.2,
                           start=None,
                           stop=None,
                           time_col='time',
                           vx_col='vx',
                           vy_col='vy',
                           speed_col='speed',
                           ang_vel_col='ang_speed',
                           spike_col='spike_count'):
    """
    Bin continuous kinematic features and discrete spike counts
    into fixed-width bins (e.g., 200 ms) for GLM analysis.

    Parameters
    ----------
    df : DataFrame
        Must contain time series columns for kinematics (vx, vy, speed,
        ang_speed) and spike counts at native resolution.
    bin_width : float
        Desired bin size in seconds (e.g., 0.2 for 200 ms).
    start, stop : float, optional
        Start/stop times for binning. Defaults to df[time_col].min/max.
    time_col : str
        Column with timestamps in seconds.
    vx_col, vy_col : str
        Column names for velocity components.
    speed_col : str
        Column for instantaneous speed (|v|).
    ang_vel_col : str
        Column for angular velocity (heading rate).
    spike_col : str
        Column for spike counts (already per-sample counts).

    Returns
    -------
    binned_df : DataFrame, shape (n_bins, n_features)
        Contains:
          - t_bin: bin centers
          - vx_bin, vy_bin: time-weighted mean velocity components
          - speed_bin: distance traveled / Δt (time-weighted mean of speed)
          - ang_speed_bin: time-weighted mean angular velocity
          - spike_count_bin: summed spike counts in bin
          - offset_log_bin: log(bin_width), for use as Poisson offset

    Notes
    -----
    - Kinematics: use **time-weighted mean** to respect irregular dt.
    - Speed: time-weighted mean(|v|) is equivalent to distance traveled / Δt.
    - Spikes: summed, since spike_count is per-sample.
    - offset_log_bin ensures GLM models rates, not raw counts.
    """
    t = df[time_col].to_numpy()
    if start is None: start = float(t[0])
    if stop  is None: stop  = float(t[-1])
    edges = np.arange(start, stop + bin_width*0.5, bin_width)

    # Velocity components: time-weighted average in each bin
    vx_bin = bin_timeseries_weighted(t, df[vx_col].to_numpy(), edges, how='mean')
    vy_bin = bin_timeseries_weighted(t, df[vy_col].to_numpy(), edges, how='mean')

    # Speed: mean(|v|) = (∫|v| dt) / Δt = distance / Δt
    speed_mean = bin_timeseries_weighted(t, df[speed_col].to_numpy(), edges, how='mean')
    speed_bin  = speed_mean

    # Angular velocity: time-weighted mean (signed); use abs(...) if you want magnitude
    ang_vel_bin = bin_timeseries_weighted(t, df[ang_vel_col].to_numpy(), edges, how='mean')

    # Spikes: sum across samples (∑ spikes in bin)
    spike_sum = bin_timeseries_weighted(t, df[spike_col].to_numpy(), edges, how='sum')

    # Bin centers for plotting / alignment
    centers = 0.5 * (edges[:-1] + edges[1:])

    out = pd.DataFrame({
        't_bin': centers,
        'vx_bin': vx_bin,
        'vy_bin': vy_bin,
        'speed_bin': speed_bin,
        'ang_speed_bin': ang_vel_bin,
        'spike_count_bin': spike_sum,
        'offset_log_bin': np.log(bin_width),
    })
    return out


## binning 3

In [ ]:
def make_binned_features_with_stops(
    df,
    bin_width=0.2,
    start=None,
    stop=None,
    # column names
    time_col='time',
    x_col='x',
    y_col='y',
    vx_col='vx',
    vy_col='vy',
    speed_col='speed',
    ang_speed_col='ang_speed',
    spike_col='spike_count',
    stop_id_col='stop_id',
    stop_onset_col='stop_id_start_time',
    # how to bin position
    pos_method='center'
):
    """
    Bin position, kinematics, and spikes into fixed-width bins,
    and also attach stop-aligned labels to each bin.

    Parameters
    ----------
    df : DataFrame
        Input samples at irregular times (native resolution). Must contain:
          - time, x, y
          - kinematics: vx, vy, speed, ang_speed
          - spike_count
          - stop_id, stop_id_start_time (for stop-aligned labeling)
    bin_width : float
        Bin width in seconds (e.g., 0.2 for 200 ms).
    start, stop : float, optional
        Start/stop times. Defaults to min/max of df[time_col].
    pos_method : {'center','start','end','mean'}
        How to compute per-bin position:
          - 'center'/'start'/'end' : interpolate position to that time
          - 'mean' : time-weighted mean x,y within the bin

    Returns
    -------
    binned : DataFrame, one row per bin, with columns:
        t_bin           : bin center (or query time used for position)
        x_bin, y_bin    : position per bin
        vx_bin, vy_bin  : time-weighted mean velocity components
        speed_bin       : time-weighted mean speed
        ang_speed_bin   : time-weighted mean angular velocity
        spike_count_bin : summed spikes per bin
        offset_log_bin  : log(bin_width), for Poisson GLM offset
        stop_id         : ID of stop active at this bin (NaN if none)
        t_rel_to_stop_bin : bin_time - stop_onset_time (NaN if no stop)
        prepost_bin     : 0 = pre-stop, 1 = post-stop (relative to onset)
    """

    # ----- Build bin edges -----
    # If start/stop not given, use data min/max.
    t = df[time_col].to_numpy()
    if start is None: start = float(t[0])
    if stop  is None: stop  = float(t[-1])
    edges = np.arange(start, stop + bin_width*0.5, bin_width)
    centers = 0.5 * (edges[:-1] + edges[1:])  # bin centers for convenience

    # ----- Bin positions -----
    # Use helper _bin_positions, which interpolates (center/start/end)
    # or computes time-weighted mean (method='mean').
    pos_binned = _bin_positions(
        df, edges,
        time_col=time_col, x_col=x_col, y_col=y_col,
        method=pos_method
    )

    # ----- Bin kinematics (time-weighted means) -----
    vx_bin = _bin_timeseries_weighted(t, df[vx_col].to_numpy(), edges, how='mean')
    vy_bin = _bin_timeseries_weighted(t, df[vy_col].to_numpy(), edges, how='mean')
    speed_bin = _bin_timeseries_weighted(t, df[speed_col].to_numpy(), edges, how='mean')
    ang_speed_bin = _bin_timeseries_weighted(t, df[ang_speed_col].to_numpy(), edges, how='mean')

    # ----- Bin spikes (sums) -----
    # Here spike_col should be "count in that native bin",
    # so summing across native samples gives total spikes per 200 ms.
    spike_count_bin = _bin_timeseries_weighted(t, df[spike_col].to_numpy(), edges, how='sum')

    # ----- Assemble the main binned DataFrame -----
    binned = pd.DataFrame({
        't_bin': pos_binned['t_bin'].to_numpy(),  # time anchor for each bin
        'x_bin': pos_binned['x_bin'].to_numpy(),
        'y_bin': pos_binned['y_bin'].to_numpy(),
        'vx_bin': vx_bin,
        'vy_bin': vy_bin,
        'speed_bin': speed_bin,
        'ang_speed_bin': ang_speed_bin,
        'spike_count_bin': spike_count_bin,
        'offset_log_bin': np.log(bin_width),  # log-binwidth for Poisson offset
    })

    # ----- Add stop-aligned labels (optional) -----
    # This requires that df has stop_id and stop_onset_col columns.
    # For each bin, we assign the most recent stop (if any) whose onset
    # is <= bin time.
    if stop_id_col in df.columns and stop_onset_col in df.columns:
        df_sorted = df[[time_col, stop_id_col, stop_onset_col]].sort_values(time_col)

        stop_ids = []
        t_rel = []
        prepost = []

        for tb in binned['t_bin']:
            # find last sample <= tb
            idx = np.searchsorted(df_sorted[time_col].to_numpy(), tb, side='right') - 1
            if idx < 0:
                # before first sample → no stop
                stop_ids.append(np.nan)
                t_rel.append(np.nan)
                prepost.append(0)
            else:
                sid = df_sorted.iloc[idx][stop_id_col]
                onset = df_sorted.iloc[idx][stop_onset_col]
                if pd.isna(sid):
                    # no stop active at this time
                    stop_ids.append(np.nan)
                    t_rel.append(np.nan)
                    prepost.append(0)
                else:
                    # inside a stop → assign labels
                    stop_ids.append(sid)
                    t_rel.append(tb - onset)      # time since stop onset
                    prepost.append(int(tb >= onset))  # 0=pre, 1=post

        binned['stop_id'] = stop_ids
        binned['t_rel_to_stop_bin'] = t_rel
        binned['prepost_bin'] = prepost

    return binned


## debug

In [ ]:
cols = [c for c in design_df.columns if c.startswith('cur_angle_cos_rc')]
design_df[cols].var().sort_values()

In [ ]:
# A) How much signal is actually in the cos block?


# B) Are those params actually ~0 and SE huge?
res.params.loc[['const'] + cols]           # param values
res.cov_params().loc[cols, cols].abs().max().max()  # largest abs entry in cov submatrix



# My Glm

## single neurons

In [ ]:

# design_df.dtypes[design_df.dtypes == "object"]
# design_df.loc[:, design_df.dtypes == "object"].head()


In [ ]:
data = pn.rebinned_y_var.copy()
spike_data = pn.rebinned_x_var['cluster_0']
dt = pn.bin_width

res, design_df, metrics, meta = glm_on_multiff.fit_multiff_glm(
    dt=dt, trial_ids=data["new_segment"],
    cur_vis=data["cur_in_memory"], nxt_vis=data["nxt_in_memory"],
    cur_dist=data["cur_ff_distance"], nxt_dist=data["nxt_ff_distance"],
    cur_angle=data["cur_ff_angle"], nxt_angle=data["nxt_ff_angle"],
    heading=None, speed=data["monkey_speed"], curvature=data["curv_of_traj"],
    spike_counts=spike_data, l2=0.0, use_trial_FE=True, cluster_se=False,
)
print("Metrics:\n", {k: (v if k != 'per_trial_deviance' else '... DataFrame ...') for k, v in metrics.items()})

In [ ]:
# res_nr = sm.GLM(spike_data, sm.add_constant(design_df),
#                 family=sm.families.Poisson(),
#                 exposure=np.full(len(design_df), float(pn.bin_width))
#                ).fit()  # non-robust (model-based) cov
# plot_glm_fit.plot_angle_kernels_with_ci(res_nr, design_df, meta, float(pn.bin_width),
#                                         base_prefix='cur_angle', show_history=True)


In [ ]:
# plot_glm_fit.plot_fitted_kernels(res, design_df, meta, data["dt"])
# plot_glm_fit.plot_angle_tuning(res, design_df, meta, data["dt"])

In [ ]:
for base_prefix in ['cur_angle', 'nxt_angle', 'cur_dist', 'nxt_dist', 'cur_vis', 'nxt_vis', 'speed', 'curvature']:
    plot_glm_fit.plot_angle_kernels_with_ci(res, design_df, meta, dt, base_prefix=base_prefix)


## across neurons

In [ ]:
cluster_cols = [col for col in pn.rebinned_x_var.columns if col.startswith('cluster_')]
cluster_nums = [int(col.split('_')[1]) for col in cluster_cols]

In [ ]:
cluster_nums

In [ ]:
res, design_df, metrics, meta = glm_on_multiff.fit_multiff_glm(
    dt=pn.bin_width, trial_ids=data["new_segment"],
    cur_vis=data["cur_in_memory"], nxt_vis=data["nxt_in_memory"],
    cur_dist=data["cur_ff_distance"], nxt_dist=data["nxt_ff_distance"],
    cur_angle=data["cur_ff_angle"], nxt_angle=data["nxt_ff_angle"],
    heading=None, speed=data["monkey_speed"], curvature=data["curv_of_traj"],
    spike_counts=spike_data, l2=0.0, use_trial_FE=True, cluster_se=False,
)

In [ ]:
results = []
designs = []
B_hist_ref = None

for cluster in cluster_nums:
    # Simulate one "neuron" worth of data (independent draws)
    trial_ids, y, stim_dict, stim_basis_dict, B_hist = simulate_spikes_with_trials(
        n_trials=12, trial_len=300, dt=dt, seed=seed + i
    )
    if B_hist_ref is None:
        B_hist_ref = B_hist

    # Build design + history for this neuron
    design_df, y_fit = build_glm_design_with_trials(
        dt=dt,
        trial_ids=trial_ids,
        stimulus_dict=stim_dict,
        stimulus_basis_dict=stim_basis_dict,
        spike_counts=y,
        history_basis=B_hist,
        extra_covariates=None,
        use_trial_FE=True,
    )

    # Fit GLM (cluster-robust by trial)
    res = fit_poisson_glm_trials(design_df, y_fit, dt, trial_ids, add_const=True, l2=0.0, cluster_se=False)
    results.append(res)
    designs.append(design_df)

meta = {"B_hist": B_hist_ref}

# Collect population history kernels
hist_df = collect_history_kernels_across_neurons(results, designs, meta, dt)

# Plot overlays + heatmap
plot_history_kernels_population(hist_df, overlay_mean=True, heatmap=True, max_overlays=50)

In [ ]:
# Suppose you looped over N neurons and saved:
results = [res_n0, res_n1, ..., res_nN]         # statsmodels results
designs = [X_n0_df, X_n1_df, ..., X_nN_df]      # matching design DataFrames
meta    = meta_from_any_single_fit              # must contain 'B_hist'
dt      = 0.01                                  # your bin size (s)

hist_df = collect_history_kernels_across_neurons(
    results, designs, meta, dt, neuron_ids=None  # or e.g. list of unit IDs
)


# Overlay individual kernels (up to max_overlays) + population mean ± 95% CI
plot_history_kernels_population(hist_df, overlay_mean=True, heatmap=False, max_overlays=60)

# Heatmap only (neuron × lag)
plot_history_kernels_population(hist_df, overlay_mean=False, heatmap=True)

# Both
plot_history_kernels_population(hist_df, overlay_mean=True, heatmap=True)




# Example data

In [ ]:
data = glm_on_multiff.simulate_multiff_trials(n_trials=8, trial_len=350, dt=0.01, seed=3)
res, design_df, metrics, meta = glm_on_multiff.fit_multiff_glm(
    dt=data["dt"], trial_ids=data["trial_ids"],
    cur_vis=data["cur_vis"], nxt_vis=data["nxt_vis"],
    cur_dist=data["cur_dist"], nxt_dist=data["nxt_dist"],
    cur_angle=data["cur_angle"], nxt_angle=data["nxt_angle"],
    heading=data["heading"], speed=data["speed"], curvature=data["curvature"],
    spike_counts=data["spike_counts"], l2=0.0, use_trial_FE=True, cluster_se=False,
)
print("Metrics:\n", {k: (v if k != 'per_trial_deviance' else '... DataFrame ...') for k, v in metrics.items()})
# plot_glm_fit.plot_fitted_kernels(res, design_df, meta, data["dt"])
# plot_glm_fit.plot_angle_tuning(res, design_df, meta, data["dt"])
plot_glm_fit.plot_angle_kernels_with_ci(res, design_df, meta, data["dt"], base_prefix='cur_angle')


In [ ]:
for key, item in data.items():
    try:
        print(key, item.shape)
    except:
        print(key, item)

# PGAM functions

## individual steps

### categorize variables

In [ ]:
pgam_inst = pgam_class.PGAMclass(pn.x_var, pn.y_var_reduced, pn.bin_width, pn.processed_neural_data_folder_path)

In [ ]:
pgam_inst.prepare_for_pgam(num_total_trials=10)

### temporal kernel

modified from PGAM_Tutorial.ipynb

In [ ]:
pgam_inst._add_temporal_features_to_model()

In [ ]:
help(gdh.smooths_handler.add_smooth)

### spatial variable

In [ ]:
pgam_inst._add_spatial_features_to_model()

### run

In [ ]:
pgam_inst.run_pgam(neural_cluster_number=10)

### post-processing

In [ ]:
pgam_inst.post_processing()

### save results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import BSpline

# Degree of spline
k = 3

# Example 1: Not enough start knots
knots1 = [0, 2, 4, 6, 8, 10, 10, 10, 10]
coeffs = np.ones(len(knots1) - k - 1)
spline1 = BSpline(knots1, coeffs, k)

# Example 2: Proper repeated start knots
knots2 = [0, 0, 0, 0, 2, 4, 6, 8, 10, 10, 10, 10]
coeffs2 = np.ones(len(knots2) - k - 1)
spline2 = BSpline(knots2, coeffs2, k)

x = np.linspace(-1, 11, 400)

plt.plot(x, spline1(x), label="Start knot once")
plt.plot(x, spline2(x), label="Start knot repeated")
plt.axvline(0, color='gray', linestyle='--', label="x = 0")
plt.legend()
plt.title("Effect of Repeating Start Knot")
plt.xlabel("x")
plt.ylabel("Spline Value")
plt.grid(True)
plt.show()


In [ ]:
pgam_inst.save_results()

In [ ]:
stop!

## iterate through all neurons

In [ ]:
pgam_inst = pgam_class.PGAMclass(pn.x_var, pn.y_var, pn.bin_width, pn.processed_neural_data_folder_path)

In [ ]:
for i in range(pn.x_var.shape[1]):
    print(f'neural_cluster_number: {i} out of {pn.x_var.shape[1]}')
    pgam_inst.streamline_pgam(neural_cluster_number=i, num_total_trials=10)